<a href="https://colab.research.google.com/github/dianalves00/6-7-edition/blob/main/Supervised%20Learning/SVM_Hyperparameter_Tuning_WineQuality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning Practice: Wine Quality Dataset (SVM)
In this notebook, you'll explore hyperparameter tuning using Support Vector Machines (SVM) on the Wine Quality dataset. You'll learn how to tune key hyperparameters, visualize results, and interpret the impact of these hyperparameters.

## Objectives:
1. Load and preprocess the Wine Quality dataset.
2. Split the data into training and test sets.
3. Perform hyperparameter tuning for SVM.
4. Visualize training vs validation performance.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Enable inline plots
%matplotlib inline

## Step 1: Load the Wine Quality Dataset
We'll use the Wine Quality dataset, which contains information about different wines and their quality ratings. You can download it from: [Wine Quality Dataset](https://www.kaggle.com/datasets/yasserh/wine-quality-dataset) and find more information, or load it directly from github.


In [2]:
# Load the dataset (you'll need to download 'winequality-red.csv')
url = 'https://raw.githubusercontent.com/samsung-ai-course/6-7-edition/refs/heads/main/Supervised%20Learning/Datasets/WineQT.csv'
data = pd.read_csv(url)

# Display the first few rows
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


### Step 1.1: Free EDA
Here you are free to explore the data in whatever it be of use to you.
* .describe()
* .info()
* plot correlation matrixes
* up to you


In [3]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
count,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000
mean,8.311111,0.531339,0.268364,2.532152,0.086933,15.615486,45.914698,0.996730,3.311015,0.657708,10.442111,5.657043,804.969379
std,1.747595,0.179633,0.196686,1.355917,0.047267,10.250486,32.782130,0.001925,0.156664,0.170399,1.082196,0.805824,463.997116
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000,0.000000
25%,7.100000,0.392500,0.090000,1.900000,0.070000,7.000000,21.000000,0.995570,3.205000,0.550000,9.500000,5.000000,411.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,37.000000,0.996680,3.310000,0.620000,10.200000,6.000000,794.000000
75%,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,61.000000,0.997845,3.400000,0.730000,11.100000,6.000000,1209.500000
max,15.900000,1.580000,1.000000,15.500000,0.611000,68.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000,1597.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
 12  Id                    1143 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 116.2 KB


## Step 2: Data Preprocessing
- Separate features and target variable.
- Split the dataset into training and testing sets.
- Optionally perform some feature engineering and assess its impact.

In [11]:
# Separate features and target

X = data.drop(columns=['quality'])
y = data["quality"]

In [12]:
# Bin target variable into 'low', 'medium', and 'high' quality for classification -> https://en.wikipedia.org/wiki/Data_binning
y = pd.cut(y, bins=[0, 5, 7, 10], labels=['low', 'medium', 'high'])
#TODO lets turn this into a simpler classification problem p.s lookup pd.cut
#p.s this will be a multi-class problem instead of a simple binary one. Class imbalance might cause more problems here..?
#More on that on the next class, but think about it.

# Split into training and testing sets
train_test_split_ratio = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=train_test_split_ratio, random_state=42)

print(f"Training set: {X_train.shape}, Testing set: {X_test.shape}")

Training set: (914, 12), Testing set: (229, 12)


## Step 3: SVM Hyperparameter Tuning
Experiment with different values for `C` (regularization parameter) and kernel types to see how they affect training and validation accuracy.

In [ ]:
# Define parameter grid for SVM
# Note: the more cross validations (cv) and the bigger the search space, the slower this can be
param_grid = {
    'C': [0.1],
    'kernel': ['linear', 'rbf'],
    'gamma': ['auto']  # Only relevant for non-linear kernels like 'rbf'
}

# Perform Grid Search
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and scores
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

In [ ]:
random_search = #TODO -> https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.RandomizedSearchCV.html
#TODO

print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validation Accuracy:", random_search.best_score_)

## Step 4: Visualize the Results
Plot the cross-validation accuracy for different values of `C` to understand its impact.

In [ ]:
# Extract results from grid search
results = pd.DataFrame(grid_search.cv_results_)

# Filter for linear and rbf kernels separately
linear_results = results[results['param_kernel'] == 'linear']
rbf_results = results[results['param_kernel'] == 'rbf']

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(linear_results['param_C'], linear_results['mean_test_score'], label='Linear Kernel', marker='o')
plt.plot(rbf_results['param_C'], rbf_results['mean_test_score'], label='RBF Kernel', marker='o')
plt.xscale('log')
plt.title('SVM: Effect of C on Accuracy')
plt.xlabel('C (Regularization Parameter)')
plt.ylabel('Cross-Validation Accuracy')
plt.legend()
plt.grid()
plt.show()

## Step 5: Evaluation

Understand precision and recall, plot confusion matrix and evaluate the model more in detail.

Note that after the hyperparameters are found we typically train the model again on all *train* dataset.

In [ ]:
#;)